In [1]:
import pandas as pd
import os
import ruptures as rpt

In [ ]:
columns = ['index', 'time','deviceid','target','direction','protocol','tool','pktloss','method','zip','isp','value','topic','annonipaddr','ipaddrchanged']
devicefulldf = pd.read_csv('/srv/data/my_shared_data_folder/internet-equity/shortened_2022.csv',names=columns)
devicefulldf.to_csv('devicefulldf.csv')

In [ ]:
date_counts['date_missing_entries'] = date_counts['count'].map(lambda count: count_threshold - count if count < count_threshold else 0)

In [ ]:
##UPDATED VERSION

class plots:
    def __init__(self, df):
        self.full_df = self.make_full_df(df)
        
    def split_time(self, df):
        df['time'] = pd.to_datetime(df['time'])
        df['date'] = df['time'].dt.date
        
        return df
        
    def missing_rows_calculator(self, df):
        count_threshold = 288
        # Count the occurrences of each date for each deviceID and target within dataframe
        date_counts = df.groupby(['date', 'deviceid','target']).count().reset_index()
        
        count_threshold = 288
        # Count the occurrences of each date for each deviceID and target within dataframe
        date_counts = df.groupby(['date', 'deviceid','target']).agg({"direction":"count"}).reset_index()
        date_counts.columns = ['date', 'deviceid','target', 'count']
        
        # Create a new column to store the threshold comparison result
        date_counts['date_missing_entries'] = date_counts['count'].map(lambda count: count_threshold - count if count < count_threshold else 0)
        df = pd.merge(df, date_counts, how = "left", on = ['date', 'deviceid','target'])
        
        return df
        
    def make_threshold(self, df):
        anomalydf = df.groupby(['deviceid','target']).value.agg(['mean', 'std']).reset_index()
        anomalydf['threshold'] = anomalydf['mean'] + anomalydf['std']*2
        anomalydf = anomalydf.drop(['mean', 'std'], axis=1)
        df = df.merge(anomalydf, on=['deviceid','target'], how='left')
        df['anomaly'] = (df["value"] >= df["threshold"])
        
        return df
        
     #Function checks for missing dates in a dataframe and add rows for those missing dates for each unique device ID present in the dataframe.
    
    def add_missing_dates(self, df):
    # Fill 'date missing' column with false for the rest of the rows in the original dataframe using
        df.loc[:, 'date missing'] = False
    #Creates a range of dates from the minimum date to the maximum date in the 'whole date' column with a frequency of one day (daily).
        date_range = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
    #Make an empty dictionary to keep track of missing dates for each device.
        tracker = {}
    #Iterates over each unique device ID in the 'deviceid' column.
        for device in df['deviceid'].unique():
    #Selects the unique dates for the current device using the boolean indexing and stores them in device_dates variable.
            device_dates = df.loc[df.loc[:, "deviceid"] == device, "date"].unique()
    #Finds the missing dates for the current device by checking which dates from date_range are not present in device_dates.
            missing_dates = date_range[~date_range.isin(device_dates)]
    #Stores the list of missing dates for the current device in the tracker dictionary.
            tracker[device] = list(missing_dates)
    #Create a list of column names from the original dataframe
        columns = list(df.columns)
    # Create empty dataframe with the same columns as the original dataframe
        add_this = pd.DataFrame(columns = columns)
    #Iterates over the tracker dictionary to access each device and its corresponding list of missing dates
        for device, date_list in tracker.items():
    #Gets the  length of rows in the new dataframe
            start = len(add_this.index)
    #Iterates over the list of missing dates for the current device.
            for i, date in enumerate(date_list):
    #Assigns the current device ID to the 'deviceid' column of the new row in the new dataframe
                add_this.loc[start + i, "deviceid"] = device
    #Assigns the current missing date to the 'whole date' column of the new row in the new dataframe.
                add_this.loc[start + i, "date"] = date
    #Assigns all the missing rows for the missing date
                add_this.loc[start + i, 'date_missing_entries'] = 288
    #Assigns new 'date missing' column to True for rows from the missing dates.
                add_this.loc[start + i, 'date missing'] = True
    
    #Concatenates the original dataframe with the new dataframe we created, which contains the rows for the missing dates.
    #The function then returns the updated DataFrame with the missing date rows added for each device.
        #return pd.concat([df, add_this])
        combined_df = pd.concat([df, add_this], sort=False)
        
        return combined_df
    
    def make_full_df(self, df):
        new = self.split_time(df)
        new = self.missing_rows_calculator(new)
        new = self.make_threshold(new)
        return self.add_missing_dates(new)
        

In [ ]:
#Function checks for missing dates in a dataframe and add rows for those missing dates for each unique device ID present in the dataframe.
def add_missing_dates(self):
#Converts the 'whole date' column to datetime format.
    df['whole date'] = pd.to_datetime(df['whole date'])
#Creates a range of dates from the minimum date to the maximum date in the 'whole date' column with a frequency of one day (daily).
    date_range = pd.date_range(start=df['whole date'].min(), end=df['whole date'].max(), freq='D')
#Make an empty dictionary to keep track of missing dates for each device.
    tracker = {}
#Iterates over each unique device ID in the 'deviceid' column.
    for device in df['deviceid'].unique():
#Selects the unique dates for the current device using the boolean indexing and stores them in device_dates variable.
        device_dates = df.loc[df.loc[:, "deviceid"] == device, "whole date"].unique()
#Finds the missing dates for the current device by checking which dates from date_range are not present in device_dates.
        missing_dates = date_range[~date_range.isin(device_dates)]
#Stores the list of missing dates for the current device in the tracker dictionary.
        tracker[device] = list(missing_dates)
#Create a list of column names from the original dataframe
    columns = list(df.columns)
# Create empty dataframe with the same columns as the original dataframe
    add_this = pd.DataFrame(columns = columns)
#Iterates over the tracker dictionary to access each device and its corresponding list of missing dates
    for device, date_list in tracker.items():
#Gets the  length of rows in the new dataframe
        start = len(add_this.index)
#Iterates over the list of missing dates for the current device.
        for i, date in enumerate(date_list):
#Assigns the current device ID to the 'deviceid' column of the new row in the new dataframe
            add_this.loc[start + i, "deviceid"] = device
#Assigns the current missing date to the 'whole date' column of the new row in the new dataframe.
            add_this.loc[start + i, "whole date"] = date
#Assigns new 'date missing' column to True for rows from the missing dates.
            add_this.loc[start + i, 'date missing'] = True
# Fill 'date missing' column with false for the rest of the rows in the original dataframe using
    df.loc[:, 'date missing'] = False
#Concatenates the original dataframe with the new dataframe we created, which contains the rows for the missing dates.
#The function then returns the updated DataFrame with the missing date rows added for each device.
    #return pd.concat([df, add_this])
    return pd.concat([df, add_this], sort=False)
data2022_serv = add_missing_dates(data2022_serv)

In [ ]:
 def add_missing_dates(self):
        self.df['date'] = pd.to_datetime(self.df['date'])
        date_range = pd.date_range(start=self.df['date'].min(), end=self.df['date'].max(), freq='D')
        tracker = {}
        for device in self.df['deviceID'].unique():
            device_dates = self.df.loc[self.df.loc[:, "deviceID"] == device, "date"].unique()
            missing_dates = date_range[~date_range.isin(device_dates)]
            tracker[device] = list(missing_dates)
        columns = list(self.df.columns)
        add_this = pd.DataFrame(columns = columns)
        for device, date_list in tracker.items():
            start = len(add_this.index)
            for i, date in enumerate(date_list):
                add_this.loc[start + i, "deviceID"] = device
                add_this.loc[start + i, "date"] = date
        self.combined_df = pd.concat([self.df, add_this])

In [ ]:
test = plots(devicefulldf)

In [ ]:
testing = test.make_full_df()

In [ ]:
groupby = testing.groupby(['date','target','deviceID'])[['value','anomaly','date_missing_entries']].sum().reset_index()
groupby.head(700)

In [ ]:
 '''
 after missing rows function!
 
 def changept_detect(self):
        algo = rpt.Pelt(model="rbf").fit(self.fulldf["value"].values)
        result = algo.predict(pen=10)
        self.fulldf['changepts'] = result '''